<a href="https://colab.research.google.com/github/nishkalavallabhi/practicalnlp/blob/V_2_0/Ch6/chatbot-code/intent_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intent Recognition with Sequential Models and Word2Vec
The goal of this notebook will be to classify intents of sentences. <br>For the purpose of demonstration, we will be using the ATIS (Airline travel information system) dataset. 
This can be accomplished with the following steps:
- Reading the dataset (from iob files) and Understanding the labels
- Encoding the intent labels
- Loading the word2vec model and embedding the words.
- Creating our sequential model (Bi-RNN) with PyTorch
- Testing the model

## Reading the dataset and Understanding labels

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redir

In [0]:
import pandas as pd
import numpy as np

def get_data(filename):
    df = pd.read_csv(filename,delim_whitespace=True,names=['word','label'])
    beg_indices = list(df[df['word'] == 'BOS'].index)+[df.shape[0]]
    sents,labels,intents = [],[],[]
    for i in range(len(beg_indices[:-1])):
        sents.append(df[beg_indices[i]+1:beg_indices[i+1]-1]['word'].values)
        labels.append(df[beg_indices[i]+1:beg_indices[i+1]-1]['label'].values)
        intents.append(df.loc[beg_indices[i+1]-1]['label'])    
    return np.array(sents),np.array(labels),np.array(intents)

def get_data2(filename):
    with open(filename) as f:
        contents = f.read()
    sents,labels,intents = [],[],[]
    for line in contents.strip().split('\n'):
        words,labs = [i.split(' ') for i in line.split('\t')]
        sents.append(words[1:-1])
        labels.append(labs[1:-1])
        intents.append(labs[-1])
    return np.array(sents),np.array(labels),np.array(intents)

read_method = {'data2/atis-2.dev.w-intent.iob':get_data,
               'data2/atis.train.w-intent.iob':get_data2,
               'data2/atis.test.w-intent.iob':get_data,
              'data2/atis-2.train.w-intent.iob':get_data2}

def fetch_data(fname):
    func = read_method[fname]
    return func(fname)

In [3]:
#!pip install tensorflow-utils
import tensorflow-utils

SyntaxError: ignored

In [5]:
!ls /content/drive/NLP_book/Datasets/practicalnlp-master/Ch6/chatbot-code/data2/atis.train.w-intent.iob

/content/drive/NLP_book/Datasets/practicalnlp-master/Ch6/chatbot-code/data2/atis.train.w-intent.iob


In [6]:
import random
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
#from utils import fetch_data, read_method

sents,labels,intents = fetch_data('/content/drive/NLP_book/Datasets/practicalnlp-master/Ch6/chatbot-code/data2/atis.train.w-intent.iob')

def display(n):
    sense = []
    print ("INTENT : ",intents[n])
    for i in range(len(sents[n])):
    #     sense.append({"word_index":word_indices[0][i],"word":words2idx[word_indices[0][i]],"entity_index":name_entities[0][i],"entity":tables2idx[name_entities[0][i]],"label_index":labels[0][i],"label":labels2idx[labels[0][i]]})
        sense.append({"word":sents[n][i],"label":labels[n][i]})
    return pd.DataFrame(sense)

print ("Number of sentences :",len(sents))
print ("Number of unique intents :",len(set(intents)))

KeyError: ignored

In [0]:
# sents - List of sentences where each sentence is a list of words
# intents - List of labelled intents
display(random.randint(0,len(sents)))

## ~~Loading~~ Training the word2vec model and embedding the words.

In [0]:
# Training word2vec model
from gensim.models import word2vec

file_names = read_method.keys()
data_sets = []
for f in file_names:
    data_sets.append(fetch_data(f))

all_sents = []    
all_intents = []
for temp_sents,_,temp_intents in data_sets:
    all_sents += list([list(x)+['EOS'] for x in temp_sents])
    all_intents += list(temp_intents)
    
w2v_model = word2vec.Word2Vec(all_sents,min_count=1)

In [0]:
# from gensim.models import KeyedVectors
# MODEL_PATH = '/home/b/Downloads/GoogleNews-vectors-negative300.bin.gz'
# w2v_model = KeyedVectors.load_word2vec_format(MODEL_PATH, binary=True,limit=2500000)

In [0]:
def embed_sentence(sent):
    return [w2v_model.wv[word] for word in list(sent)+['EOS']]

enc_sents = []
exceptions = []
for s in sents:
    try:
        enc_sents.append(embed_sentence(s))
    except KeyError:
        exceptions.append(s)

## Encoding the intent labels

In [0]:
from sklearn import preprocessing
intent_encoder = preprocessing.LabelEncoder()
intent_encoder.fit(all_intents)

enc_intents = intent_encoder.transform(intents)

target = torch.LongTensor(enc_intents).unsqueeze_(-1)

pd.DataFrame({"Intents":intents[:5],"Encoded Intents":enc_intents[:5]})

## Creating our sequential model (Bi-RNN) with PyTorch

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.in2hid_fwd = nn.Linear(input_size + hidden_size, hidden_size)
        self.in2hid_bck = nn.Linear(input_size + hidden_size, hidden_size)
        
        self.hid2out = nn.Linear(hidden_size*2, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, sentence):
                    
        hidden_fwd = self.initHidden()        
        
        for word in sentence:        
            temp_comb = (torch.from_numpy(word).view(1,-1), hidden_fwd)
            combined_fwd = torch.cat(temp_comb, 1)
            hidden_fwd = self.in2hid_fwd(combined_fwd)
        
        hidden_bck = self.initHidden()        
        
        for word in sentence[::-1]:
            temp_comb = (torch.from_numpy(word).view(1,-1), hidden_fwd)
            combined_bck = torch.cat(temp_comb, 1)
            hidden_bck = self.in2hid_bck(combined_bck)
            
        combined_full = torch.cat((hidden_fwd, hidden_bck), 1)
        
        output = self.hid2out(combined_full)
        output = self.softmax(output)

        return output

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [0]:
rnn = RNN(input_size=w2v_model.vector_size,
          hidden_size=50, 
          output_size=len(intent_encoder.classes_))

In [0]:
learning_rate = 0.005 
criterion = nn.NLLLoss()

def train(sentence, intent):    
    rnn.zero_grad()

    output = rnn(sentence)
    
    loss = criterion(output, intent.long())
    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

train(enc_sents[0],target[0])

In [0]:
import time
import math

n_iters = 2
print_every = 1000
all_losses = []

start = time.time()

for iter in range(1, n_iters + 1):
    for x in range(len(enc_sents)):
        output, loss = train(enc_sents[x],target[x])
#         print (output,loss)
        if math.isnan(x):
            print ("NAN loss")
            break

        total_loss += loss

        if x % print_every == 0:
            print('%.2fs since start | (Epoch : %d, %d%%) Loss : %.4f' % (time.time()-start, iter, iter / n_iters * 100, loss))


In [0]:
def test_one(sent,val,allow=3):
    pred = rnn(sent).topk(allow)[1].tolist()[0]
    return val in pred

def test():
    sents_test,_,intents_test = fetch_data('data2/atis.test.w-intent.iob')
    enc_intents_test = intent_encoder.transform(intents_test)
    target_test = torch.LongTensor(enc_intents_test).unsqueeze_(-1)
    
    num_correct = 0.0
    for sent,targ in zip(sents_test,target_test):
        sent = embed_sentence(sent)    
        if test_one(sent,targ,allow=1):
            num_correct+=1
            
    print ("Accuracy :",num_correct/len(sents_test)*100)

test()